Here we:
- Generate P2,P3,P4,P5,P6

Future Tasks:

- Can use the code here to get higher MSA functions, where we avoid generating all of Pn.
We do this by, if for example we want $g_n \in \rho^{-1}(g_{n-1})$ we need to make choices of which $g_i \in \rho^{-1}(g_{i-1})$ for $2 \le i \le n-1$ to take.

# Functions

In [67]:
import itertools
import pandas as pd
import numpy as np
import pickle
import time

#General use functions:
def set_to_string(s):
    s = list(s)
    s.sort()
    # Convert the list of numbers to a list of strings
    my_list=s
    my_list = [str(i) for i in my_list]
    # Use the join() function to convert the list of strings to a single string
    result = ''.join(my_list)
    return result
def string_to_set(s):
    my_list = list(s)
    # Convert the list of strings to a list of integers
    my_list = list(map(int,my_list))
    # Convert the list to a set
    my_set = set(my_list)
    return my_set

def get_powerset_str(n):
    s=set(range(1,n+1))
    power_set = [set(x)  for r in range(len(s) + 1) for x in itertools.combinations(s, r)]
    power_set_str=[set_to_string(x) for x in power_set if len(x)>0]

    return power_set_str

def get_n_np1_powersets(n):
    s=set(range(1,n+1))
    power_set = [set(x)  for r in range(len(s) + 1) for x in itertools.combinations(s, r)]
    power_set_str=[set_to_string(x) for x in power_set if len(x)>0]

    np1=n+1
    sp1=set(range(1,n+2))
    power_set_np1 = [set(x)  for r in range(np1+1 + 1) for x in itertools.combinations(sp1, r)]
    power_set_str_np1=[set_to_string(x) for x in power_set_np1 if len(x)>0]

    return power_set_str,power_set_str_np1
def sorted_frozenset(s):
    t=sorted(s)
    r=frozenset(t)
    return r

# Functions for primitive closed sets:
def fmin(A,B,f_dict):
    #A included in B
    # global f_dict
    
    valA=f_dict[A]
    valB=f_dict[B]
    compBA=set_to_string(string_to_set(B)-string_to_set(A)) # B minus A
    if len(compBA)==0: # to avoide when taking the complement gives empty set
        return False
    valBA=f_dict[compBA]
    if valB==valA+valBA:
        return True
    else:
        return False
def fmax(A,B,f_dict):
    #A included in B
    # global f_dict
    
    valA=f_dict[A]
    valB=f_dict[B]
    compBA=set_to_string(string_to_set(B)-string_to_set(A)) # B minus A
    
    #A has to be a subset of B first
    if len(compBA)==0: # to avoide when taking the complement gives empty set
        return False
    valBA=f_dict[compBA]
    if valB==valA+valBA+1:
        return True
    else:
        return False
def next_j(ji,f_dict):
    #ji is the ith set on constructing the closure. 
    # print("ji",ji)
    # global f
    my_list=[]    
    for x in power_set_str:
        for y in ji:
            if fmin(y,x,f_dict) and string_to_set(y).issubset(string_to_set(x)):
                # print("y is",y)
                my_list.append(x)
                # print(f"{x} containing {y} f min ")
    for x in power_set_str:
        for y in ji:
            if fmax(x,y,f_dict) and string_to_set(x).issubset(string_to_set(y)):
                my_list.append(x)
    jip1=set(my_list).union(ji)
    return jip1    
def rec_j(T,S,f_dict):
    # T=Ji and S=Ji-1, |T|\ge |S|.
    C=string_to_set(T)-string_to_set(S) #T \S
    C={str(x) for x in C}    
    # print("C",C,f"S term {S}",f"T term {T}")
    if len(C)==0: #Check if closed set
        return T # return closed set
    if len(C)>0:
        U=next_j(T,f_dict) #next_j(T)
        Z=rec_j(U,T,f_dict)
    # print(T)
    return Z
def get_barj(J,f_dict):
    j0=J
    j1=next_j(j0,f_dict)
    T=rec_j(j1,j0,f_dict)
    return T

# Main: making use of primitives to get all closed sets:
def Recur(Ibar,indicators,Eg,Memory,dict_PC):
    """
    inputs:
    returns:
    """
    # global dict_PC
        
    # To save recalulating we uses this series of checks, if fails checks end recursion turn.
    
    if indicators in Memory:# Memortisation for preventing repeated calculations.
        # print("Fails Memory check")
        # print(f"\n At this failure we have \n Eg: {len(Eg)} \n Memory: {len(Memory)} \n")
        return Eg,Memory
    
    y=indicators[-1]
    # print(f"term taken for primitive closed set we are unioning {y}")
    P_y=dict_PC[y] # the primitive closed set wrt y
    Jbar= Ibar.union(P_y)
    
    if len(Jbar)==len(power_set_str): # We always have the power set in Eg
        # print(f"Fails as {Jbar} == poweset")
        # print(f"\n At this failure we have \n Eg: {len(Eg)} \n Memory: {len(Memory)} \n")
        return Eg,Memory
              
    if Jbar in Eg:
        # print(f"Fails as {Jbar} in Eg")
        # print(f"\n At this failure we have \n Eg: {len(Eg)} \n Memory: {len(Memory)} \n")
        return Eg,Memory
    else:
        # print("Passes Checks \n")
        #Add the new data to memory
        set_indicators=set([sorted_frozenset(tuple(indicators)),])
        Memory=Memory.union(set_indicators)
        
        Jbar_union=set([sorted_frozenset(Jbar),])#Correct format for union
        #Add new data to Eg
        Eg=Eg.union(Jbar_union) # will ge an issue with where defines and globals.

        #Move onto the next recursion.
        power_complment=set(power_set_str)-Jbar
        for w in power_complment: # pick one in the complement to to continus exhaustive method
            new_indicators=indicators+(w,) #want to add it at end
            Eg,Memory=Recur(Jbar,new_indicators,Eg,Memory,dict_PC)
            # print(f"Done with {new_indicators} \n")
                
    return Eg,Memory # will also be an issue.
def get_Eg(dict_PC,power_set_str,set_PC):
    
    Eg={frozenset(power_set_str)}.union(set_PC) #empty set is an issue. #Main set we wish to construct
    Memory={sorted_frozenset(("0"))}    # contains indicators which are tuples of x \in powerset for primitives sets.
    
    for x in power_set_str:
        Ibar=dict_PC[x] #primitive closed set
        power_complment=set(power_set_str)-Ibar
        for y in power_complment: #exhaustively getting all closed sets.
            indicators=(x,y)
            set_indicators=set([sorted_frozenset(tuple(indicators)),])
            Memory=Memory.union(set_indicators)            
            Eg,Memory=Recur(Ibar,indicators,Eg,Memory,dict_PC)
            
    return Eg

# Creating functions in $P_{n+1}$ from Eg for this f_dict.
def add_n_p_1_to_string(string,n):
    #Adds n+1 to strings subsets
    np1string=string+(f"{n+1}")
    return np1string

def get_extension_of_f(f_dict,extender,power_set_str,n):
    # Need to attatch n+1 to strings to build extension of function by epsilon.

    # Build the extension of f by epsilon.
    
    # Add the (key,values) of f_dict
    builder_ext_0=[(k, f_dict[k]) for k in f_dict]

    #We add 1 to the function value of f_dict for x in extender
    builder_ext_p1=[(add_n_p_1_to_string(x,n),f_dict[x]+1) for x in list(extender)]

    #We add 0 to the function value of f_dict for x NOT in extender
    compl_extender=set(power_set_str) - set(extender)
    builder_ext_p2=[(add_n_p_1_to_string(x,n),f_dict[x]+0) for x in list(compl_extender)]
    
    # We include f({n+1})=0
    builder_ext_np1=[(add_n_p_1_to_string("",n),int(0)),]


    #Compile together.
    extension_f_dict={**dict(builder_ext_0),**dict(builder_ext_p1),**dict(builder_ext_p2),**dict(builder_ext_np1)}
    
    #Example
    # get_extension_of_f(frozenset({'13', '23'}))

    return extension_f_dict
def empty_case_get_extension_of_f(f_dict,extender,power_set_str,n):
    # Need to attatch n+1 to strings to build extension of function by epsilon.

    # Build the extension of f by epsilon.
    
    # Add the (key,values) of f_dict
    builder_ext_0=[(k, f_dict[k]) for k in f_dict]

    builder_ext_p2=[(add_n_p_1_to_string(x,n),f_dict[x]+0) for x in list(power_set_str)]

    builder_ext_np1=[(add_n_p_1_to_string("",n),int(0)),]

    
    #Compile together.
    extension_f_dict={**dict(builder_ext_0),**dict(builder_ext_p2),**dict(builder_ext_np1)}
    
    return extension_f_dict

def Build_Pn_1(list_df_Pn,power_set_str,n):
    # Returns:packets of extensions for each f_dict from Pn, can check those with max size of closed sets (later)
    
    # global power_set_str
    
    P_n1=[]
    for f_dict in list_df_Pn:

        # Store primitive closed sets 

        PC=[(x,get_barj({x},f_dict)) for x in power_set_str]
        dict_PC=dict(PC)
        set_PC=[sorted_frozenset(get_barj({x},f_dict)) for x in power_set_str]

        #Get all closed sets for extensions for f_dict
        Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
        Eg.append("empty")

        #Build extensions for f_dict
        extension_of_f_dict=[]
        for term in Eg:
            if term =="empty":
                extension_of_f_dict.append(empty_case_get_extension_of_f(f_dict,term,power_set_str,n))
            else:
                extension_of_f_dict.append(get_extension_of_f(f_dict,term,power_set_str,n))

        #We record packets of extensions where we take +1
        P_n1.append(extension_of_f_dict)
    
    return P_n1

def extend_sets_to_Pnp1(P_n1):
    #Return the data frame of Pn+1
    
    final_P_n1=[]
    for ls in P_n1:
        final_P_n1=final_P_n1+ls # Adding all the extension packets together.
    df=pd.DataFrame(final_P_n1)
    # df = df.reindex(columns=power_set_str_np1)
    
    return df

# This function returns Pn+1


def get_Pnp1(Pn,n):

    #inputs
    # Pn and n.

    #New data created
    np1=n+1
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)

    #Load Pn as a list of dictionaries.
    list_df_Pn=Pn.to_dict(orient='records')

    #Create Pn+1 and hold as dataframe.
    P_n1=Build_Pn_1(list_df_Pn,power_set_str,n)
    Pnp1=extend_sets_to_Pnp1(P_n1)

    Pnp1 = Pnp1.reindex(columns=power_set_str_np1)

    inspect_Pn1(Pnp1)

    return Pnp1

# Analysis of Pn+1

def string_cols(df):
    
    #Changing them to string format!only
    #reorders the columns of dataframe to powerset list order.
    
    cols=df.columns
    rnamer=dict([(cols[i],str(cols[i])) for i in range(0,len(cols))])
    df=df.rename(columns=rnamer)
    
    return df

def inspect_Pn1(df):
    print("Number of functions:",len(df.index))
    # df=df.unique()

    duplicate_rows = df[df.duplicated()]
    print("Are there duplicates?",duplicate_rows.shape, "if (0,-) then no common rows")

    # df.head()
    return

def check_df_duplicates(df1,df2,How,value):
    #Compare df1,df2 if that common rows
    
    # how is either "right" or "left"
    # for left gives those in df1 and says True or false if also in df2
    
    #     example
    #     df1 = pd.DataFrame({'team' : ['A', 'B', 'C', 'D', 'E'], 
    #                     'points' : [12, 15, 22, 29, 24]}) 
    #  #create second DataFrame
    # df2 = pd.DataFrame({'team' : ['A', 'D', 'F', 'G', 'H'],
    #                     'points' : [12, 29, 15, 19, 10]})
    
    #merge two dataFrames and add indicator column
    all_df = pd.merge(df1, df2, how=How, indicator='exists')

    #add column to show if each row in first DataFrame exists in second
    all_df['exists'] = np.where(all_df.exists == 'both', True, False)

    #view updated DataFrame
    # print (all_df)

    m=all_df.loc[all_df['exists'] == True]
    
    #Number of terms common
    num_items = m.loc[m['exists'] == value].shape[0]
    print(f"Number of those in both: {num_items} \n")

    return m    

def is_valid_function(f,power_set_str):
    # define the condition to check for each function
    Pn=[string_to_set(x) for x in power_set_str] #convert to sets
    
    # f(T) must equal 0 for elements T in Pn with size 1

    for T in Pn:
        if any(f[set_to_string(T)] != 0 for T in Pn if len(T) == 1):
            return False
    # for any I, J in Pn such that I and J have no common elements, 1 > f(I U J) - f(I) - f(J) or 0 = f(I U J) - f(I) - f(J)
    for I in Pn:
        for J in Pn:
            if len(I.intersection(J)) == 0 and f[set_to_string(I.union(J))] - f[set_to_string(I)] - f[set_to_string(J)] < 0 or f[set_to_string(I.union(J))] - f[set_to_string(I)] - f[set_to_string(J)] > 1:
            # for any I, J in Pn such that I and J have no common elements, 0 <= f(I U J) - f(I) - f(J) <= 1
                return False
    return True

def check_functs_MSA(df,power_set_str):

    num_rows=df.shape[0]

    bad_f_in_df=[] #indices of functions fail to be MSA in df
    for i in range(0,num_rows):
        f=df.loc[i].to_dict()
        if  is_valid_function(f,power_set_str)==False:
            bad_f_in_df.append(f)
    
    bad_df=pd.DataFrame(bad_f_in_df)
    
    return bad_df #indices of functions that are not MSA

# P2 extending to P3

In [2]:
#Easy to know $P_2$
P2=pd.DataFrame([{'1': 0, '2': 0, '12': 0},{'1': 0, '2': 0, '12': 1}])

n=2
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P3=get_Pnp1(P2,2)
print(P3)

Number of functions: 10
Are there duplicates? (0, 7) if (0,-) then no common rows
   1  2  3  12  13  23  123
0  0  0  0   0   1   0    1
1  0  0  0   0   0   0    1
2  0  0  0   0   1   1    1
3  0  0  0   0   0   1    1
4  0  0  0   0   0   0    0
5  0  0  0   1   0   1    1
6  0  0  0   1   1   1    2
7  0  0  0   1   1   0    1
8  0  0  0   1   1   1    1
9  0  0  0   1   0   0    1


# P3 extending to P4

In [3]:
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P4=get_Pnp1(P3,3)

Number of functions: 154
Are there duplicates? (0, 15) if (0,-) then no common rows


All the functions in P4 are MSA? Yes

In [5]:
power4=get_powerset_str(4) #powerset on 4 elements
check_functs_MSA(P4,power4).shape
#Yes (0, 0)

(0, 0)

# P4 extending to P5

In [4]:
n=4
power_set_str,power_set_str_np1=get_n_np1_powersets(n) #gives powerset on 4 and 5 elements respectively
P5=get_Pnp1(P4,4)

Number of functions: 10334
Are there duplicates? (0, 31) if (0,-) then no common rows


We check if all functions are MSA: Yes they are

In [ ]:
# bad_P5=check_functs_MSA(P5,power_set_str_np1) #.shape #(36232, 31) fail to be MSA apparently
# bad_P5.shape # (0, 0)

# P5 extending to P6: Too long

In [ ]:
# n=5
# power_set_str,power_set_str_np1=get_n_np1_powersets(n) #gives powerset on 4 and 5 elements respectively

# P6=get_Pnp1(P5,5)

## Build P6 by extending all parts of P5 (Computationally easier)

As P6=get_Pnp1(P5,5) takes too long we break the construction into parts.

As a result we get the function get_Pnp1_for_single_funct that allows one to obtain functions for higher Pn (I have not written this but it is easy to do if wanted).

During this process we record the number of extension each function in P5 has, this will be used to give the sum

$$|P_6|=\sum_{g \in P_5} |\rho^{-1}(g)|$$

In [31]:
def get_Eg_for_single_g(df,numb,power_set_str):
    #Obj: puts Eg into correct format to check if topolgoy
    #Inputs: df=Pn, numb used to get a specific function from df
    
    # Pick function in Pn
    g=df.iloc[numb].to_dict()

    #get primitive closed sets
    PC=[(x,get_barj({x},g)) for x in power_set_str]
    dict_PC=dict(PC)
    set_PC=[sorted_frozenset(get_barj({x},g)) for x in power_set_str]

    #Get all closed sets for extensions for g
    Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
    Eg.append(frozenset()) #rembmer the empty set
    return Eg

def get_Eg_column_Pn(df):
    df=df.copy()

    # Create a new column called "new_column"
    df['Eg_Size'] = None

    # Insert data into the new column
    for i, row in df.iterrows():    
        Eg=get_Eg_for_single_g(df,i,power_set_str)
        df.at[i, 'Eg_Size'] = int(len(Eg))

    return df

def eg_column_Build_Pn_1(list_df_Pn,power_set_str,n):
    
    """
    Obj:Use Build_Pn_1 and modify change to include Eg size column in functions
    """
    
    # Returns:packets of extensions for each f_dict from Pn, can check those with max size of closed sets (later)
    
    P_n1=[]
    for index,f_dict in enumerate(list_df_Pn):

        # Store primitive closed sets 
        PC=[(x,get_barj({x},f_dict)) for x in power_set_str]
        dict_PC=dict(PC)
        set_PC=[sorted_frozenset(get_barj({x},f_dict)) for x in power_set_str]

        #Get all closed sets for extensions for f_dict
        Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
        Eg.append("empty")
        
        Eg_col_kvpair={"Eg_Size":len(Eg)}

        #Build extensions for f_dict
        extension_of_f_dict=[]
        for term in Eg:
            if term =="empty":                    
                funct=empty_case_get_extension_of_f(f_dict,term,power_set_str,n) #dictionary
                funct.update(Eg_col_kvpair)
                extension_of_f_dict.append(funct)
            else:
                funct=get_extension_of_f(f_dict,term,power_set_str,n)
                funct.update(Eg_col_kvpair)
                extension_of_f_dict.append(funct)

        #We record packets of extensions where we take +1
        P_n1.append(extension_of_f_dict)
    
    return P_n1

def get_Pnp1_for_single_funct(Pn,n):
    
    #inputs
    # Pn:dataframe and n.
    
    """
    Returns a dataframe of extensions in Pn+1 for dataframe of functions in Pn.
    """
    """
    #Example
    n=3
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)
    # P3

    funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]
    Pn=pd.DataFrame(funct_list)

    #The following should be functions in P4 that are extendions of functions in funct_list
    f_P3=get_Pnp1_for_single_funct(Pn,n)
    print(f_P3)
    """

    #New data created
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)

    #Load Pn as a list of dictionaries.
    list_df_Pn=Pn.to_dict(orient='records')

    #Create Pn+1 and hold as dataframe.
    P_n1=eg_column_Build_Pn_1(list_df_Pn,power_set_str,n)
    Pnp1=extend_sets_to_Pnp1(P_n1) #<---- ISSUE
    
    power_set_str_np1
    
    columns=list(power_set_str_np1)+["Eg_Size"]
    Pnp1 = Pnp1.reindex(columns=columns)


    inspect_Pn1(Pnp1)

    return Pnp1

def Pnp1_from_Pn_part(df,n,calc_start,calc_end):
    # df where we are extending from
    # calc_start=0
    # calc_end=5

    rows=range(calc_start,calc_end) #What functions we are extending
    Pn=df.iloc[rows] #takes database of rows 0,1..,calc_up_to

    # print("The functions we are extending")
    # print(Pn,"\n")

    part_Pn=get_Pnp1_for_single_funct(Pn,n) #Extensions of Pn
    # print(part_Pn.shape)
    
    strt=f"P{n+1}_parts\{calc_start}_{calc_end}_Part_fromP{n}.xlsx"
    part_Pn.to_excel(strt) #Stores in excel
    return

### Example of functions used and process for P3:

#### Single function extension example

Here we see get_Pnp1_for_single_funct in action for P3.

In [17]:
#Testing functions example
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
# P3

# You can take subsets of rows as a database, then extend. Better than below
funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]
Pn=pd.DataFrame(funct_list)
print("The functions we are extending")
print(Pn)

#The following should be functions in P4 that are extendions of functions in funct_list
f_P3=get_Pnp1_for_single_funct(Pn,n)
print(f_P3)

The functions we are extending
   1  2  3  12  23  13  123
0  0  0  0   0   0   0    0
Number of functions: 19
Are there duplicates? (0, 16) if (0,-) then no common rows
    1  2  3  4  12  13  14  23  24  34  123  124  134  234  1234  Eg_Size
0   0  0  0  0   0   0   1   0   0   1    0    1    1    1     1       19
1   0  0  0  0   0   0   1   0   0   0    0    1    1    1     1       19
2   0  0  0  0   0   0   0   0   0   0    0    0    1    0     1       19
3   0  0  0  0   0   0   1   0   1   0    0    1    1    1     1       19
4   0  0  0  0   0   0   0   0   0   0    0    1    1    1     1       19
5   0  0  0  0   0   0   1   0   0   0    0    1    1    0     1       19
6   0  0  0  0   0   0   0   0   1   0    0    1    0    1     1       19
7   0  0  0  0   0   0   0   0   1   1    0    1    1    1     1       19
8   0  0  0  0   0   0   1   0   1   1    0    1    1    1     1       19
9   0  0  0  0   0   0   0   0   0   0    0    1    0    1     1       19
10  0  0  0  0  

#### Building P4 from P3 example

In [ ]:
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
# P3
df=P3 #Total dataframe

In [ ]:
calc_start=0
calc_end=5

rows=range(calc_start,calc_end) #What functions we are extending
Pn=df.iloc[rows] #takes database of rows 0,1..,calc_up_to

print("The functions we are extending")
print(Pn,"\n")

part_Pn=get_Pnp1_for_single_funct(Pn,n) #Extensions of Pn
print(part_Pn)
strt=f"P3_parts_example\{calc_start}_{calc_end}_PartP4.xlsx"
part_Pn.to_excel(strt) #Stores in excel

In [ ]:
calc_start=5
calc_end=10

rows=range(calc_start,calc_end) #What functions we are extending
Pn=df.iloc[rows] #takes database of rows 0,1..,calc_up_to

print("The functions we are extending")
print(Pn,"\n")

part_Pn=get_Pnp1_for_single_funct(Pn,n) #Extensions of Pn
print(part_Pn)
strt=f"P3_parts_example\{calc_start}_{calc_end}_PartP4.xlsx"
part_Pn.to_excel(strt) #Stores in excel

 We now merge the parts together.

In [ ]:
P4_part1=pd.read_excel('P3_parts_example/0_5_PartP4.xlsx', index_col=0) 
P4_part2=pd.read_excel('P3_parts_example/5_10_PartP4.xlsx', index_col=0) 

merged_df = pd.concat([P4_part1, P4_part2], ignore_index=True)

merged_df

### Building P6 from P5

You can run the following cell to get the data for P6 (take about 4hrs)

In [19]:
# Part builder cell for P5 to P6 #At this rate it will take 1.43hrs<time<6hrs to run this calculation

n=5
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
df=P5 #Total dataframe

#### Test case

In [20]:
start_perf,start_process = time.perf_counter(),time.process_time()
Pnp1_from_Pn_part(df,n,0,100)
end_perf,end_process = time.perf_counter(),time.process_time()

print(f"Perf timer {end_perf-start_perf}") #This method returns the time in seconds.
print(f"Process timer {end_process-start_process}") #measures the time the process takes, including time that the process is blocked

Number of functions: 47417
Are there duplicates? (0, 64) if (0,-) then no common rows
Perf timer 107.10247100000004
Process timer 73.859375


#### Running to construct P6

Constructs a series of 101 xlxs files to store dataframe of parts of P6

In [21]:
def runner(start,end): 
    start_perf,start_process = time.perf_counter(),time.process_time()
    Pnp1_from_Pn_part(df,n,start,end)
    end_perf,end_process = time.perf_counter(),time.process_time()

    print(f"Perf timer {end_perf-start_perf}") #This method returns the time in seconds.
    print(f"Process timer {end_process-start_process}") #measures the time the process takes, including time that the process is blocked
    return

In [ ]:
for i in range(0, 10300, 100):
    start=i
    end=start+100
    runner(start,end)

In [23]:
calc_start=10300
calc_end=10334

rows=range(calc_start,calc_end) #What functions we are extending
Pn=df.iloc[rows] #takes database of rows 0,1..,calc_up_to

part_Pn=get_Pnp1_for_single_funct(Pn,n) #Extensions of Pn
strt=f"P6_parts\{calc_start}_{calc_end}_Part_fromP5.xlsx"
part_Pn.to_excel(strt) #Stores in excel

Number of functions: 13087
Are there duplicates? (0, 64) if (0,-) then no common rows


### Analysis P6

Here we will finally get $|\tilde{P}_{6}|$. By loading all files in P6_parts and taking the shape.
As part of this calculation we can also decompose this number into the sum,

$$|P_6|=\sum_{g \in P_5} |\rho^{-1}(g)|\cdot 1_{g}$$

Note $|Eg|=|\rho^{-1}(g)|$ by construction.

The following loads the data for the parts of $P6$ (takes about 1.5hrs)

In [4]:
#Store number of rows for each data packet of P6
P6_size_list=[]
Eg_numb_functs_init_counter=[]

#Main Body of calculations
for i in range(0, 10300, 100):
    #Stores number for this packet in P6_size_list
    start=i
    end=start+100
    P6_part=pd.read_excel(f'P6_parts/{start}_{end}_Part_fromP5.xlsx', index_col=0)
    print(f'P6_parts/{start}_{end}_Part_fromP5.xlsx')
    row_numb=P6_part.shape[0]
    P6_size_list.append(row_numb)

    # In this packet gets Eg_Size and get counter for eg sizes and number of functions in Eg_numb_functs_init_counter
    values = P6_part['Eg_Size'].value_counts(ascending=True).keys().tolist()
    counts = P6_part['Eg_Size'].value_counts(ascending=True).tolist()
    zipped=list(zip(values,counts))
    Eg_numb_functs_init_counter.extend(zipped)
    
# It remains to do 10300-10334
#Stores number for this packet in P6_size_list
start,end=(10300,10334)
P6_part=pd.read_excel(f'P6_parts/{start}_{end}_Part_fromP5.xlsx', index_col=0)
print(f'P6_parts/{start}_{end}_Part_fromP5.xlsx')
row_numb=P6_part.shape[0]
P6_size_list.append(row_numb)

# In this packet gets Eg_Size and get counter for eg sizes and number of functions in Eg_numb_functs_init_counter
values = P6_part['Eg_Size'].value_counts(ascending=True).keys().tolist()
counts = P6_part['Eg_Size'].value_counts(ascending=True).tolist()
zipped=list(zip(values,counts))
Eg_numb_functs_init_counter.extend(zipped)

#output-------------------------------------------------------------------------

#Final answer
P6_size=sum(P6_size_list)
print(f"There are {P6_size} functions in P6 \n")

P6_parts/0_100_Part_fromP5.xlsx
P6_parts/100_200_Part_fromP5.xlsx
P6_parts/200_300_Part_fromP5.xlsx
P6_parts/300_400_Part_fromP5.xlsx
P6_parts/400_500_Part_fromP5.xlsx
P6_parts/500_600_Part_fromP5.xlsx
P6_parts/600_700_Part_fromP5.xlsx
P6_parts/700_800_Part_fromP5.xlsx
P6_parts/800_900_Part_fromP5.xlsx
P6_parts/900_1000_Part_fromP5.xlsx
P6_parts/1000_1100_Part_fromP5.xlsx
P6_parts/1100_1200_Part_fromP5.xlsx
P6_parts/1200_1300_Part_fromP5.xlsx
P6_parts/1300_1400_Part_fromP5.xlsx
P6_parts/1400_1500_Part_fromP5.xlsx
P6_parts/1500_1600_Part_fromP5.xlsx
P6_parts/1600_1700_Part_fromP5.xlsx
P6_parts/1700_1800_Part_fromP5.xlsx
P6_parts/1800_1900_Part_fromP5.xlsx
P6_parts/1900_2000_Part_fromP5.xlsx
P6_parts/2000_2100_Part_fromP5.xlsx
P6_parts/2100_2200_Part_fromP5.xlsx
P6_parts/2200_2300_Part_fromP5.xlsx
P6_parts/2300_2400_Part_fromP5.xlsx
P6_parts/2400_2500_Part_fromP5.xlsx
P6_parts/2500_2600_Part_fromP5.xlsx
P6_parts/2600_2700_Part_fromP5.xlsx
P6_parts/2700_2800_Part_fromP5.xlsx
P6_parts/2800

We get Eg_numb_functs_init_counter as an output also this records the decomposition of the sum $|P6|$. Eg_numb_functs_init_counter consists of (x,y) where x=|Eg| and y= x \cdot the number of times x appeared in that packet. Therefore it is neccsary to sum to get (x,Y) where Y isx \cdot the number of times x appeared in all packets. To get the number of times x appears it is necessary to divide Y by x.

In [38]:
#He now obtain Eg_numb_functs from Eg_numb_functs_init_counter

# This is a List of tuples where have  joined e.g (13, 6),(13, 19) to (13, 25) from Eg_numb_functs_init_counter

from collections import defaultdict

sums = defaultdict(int)

for first, second in Eg_numb_functs_init_counter:
    sums[first] += second

Eg_numb_functs = [(first, second) for first, second in sums.items()]
Eg_numb_functs = sorted(Eg_numb_functs, key=lambda x: x[0])#order so in increasing with respect to eg size

#Build a string using a for loop, using the tuples in Eg_numb_functs
f1,f2=Eg_numb_functs[0]

f2_div=int(f2/f1)

stringer=f'{f1}*{f2_div}'

for item in Eg_numb_functs[1:]: #first item in stringer already
    eg,numb=item #eg,numb not all the same
    num_div=int(numb/eg)
    s=f'+{eg}*{num_div}'
    stringer += s
    
#Output
print(f"Which decomposes as follows:\n {P6_size}={stringer}")

Which decomposes as follows:
 5399325=114*125+121*65+126*237+138*112+158*248+162*247+165*132+169*117+173*132+178*122+182*106+183*132+187*24+192*134+194*123+208*144+209*22+216*234+217*126+218*61+221*125+227*22+229*307+230*250+232*120+233*119+236*122+241*30+242*102+247*61+248*53+250*114+251*59+260*184+268*106+269*64+271*186+273*118+274*56+278*52+279*41+281*136+284*112+289*67+293*133+294*37+300*133+305*67+313*123+324*123+325*132+326*101+329*181+338*120+343*59+345*113+356*19+381*121+390*44+397*107+419*20+420*46+421*23+422*39+429*64+433*35+436*128+437*23+455*64+477*53+496*48+502*16+509*38+518*12+520*107+524*20+531*36+538*76+539*62+553*36+555*66+566*144+578*61+605*62+607*66+613*11+640*28+659*30+759*48+765*21+781*108+802*119+814*42+881*42+938*8+974*135+977*45+983*135+1002*45+1026*126+1029*63+1033*63+1068*96+1121*108+1145*22+1178*39+1189*39+1209*44+1212*11+1224*66+1257*44+1312*11+1322*32+1377*36+1409*16+1421*10+1450*54+1511*20+1666*21+1753*24+2110*42+2193*95+2285*4+2290*54+3544*12+3673*42+3820

In [39]:
#Check output calculates 5399325 #Yes
s=114*125+121*65+126*237+138*112+158*248+162*247+165*132+169*117+173*132+178*122+182*106+183*132+187*24+192*134+194*123+208*144+209*22+216*234+217*126+218*61+221*125+227*22+229*307+230*250+232*120+233*119+236*122+241*30+242*102+247*61+248*53+250*114+251*59+260*184+268*106+269*64+271*186+273*118+274*56+278*52+279*41+281*136+284*112+289*67+293*133+294*37+300*133+305*67+313*123+324*123+325*132+326*101+329*181+338*120+343*59+345*113+356*19+381*121+390*44+397*107+419*20+420*46+421*23+422*39+429*64+433*35+436*128+437*23+455*64+477*53+496*48+502*16+509*38+518*12+520*107+524*20+531*36+538*76+539*62+553*36+555*66+566*144+578*61+605*62+607*66+613*11+640*28+659*30+759*48+765*21+781*108+802*119+814*42+881*42+938*8+974*135+977*45+983*135+1002*45+1026*126+1029*63+1033*63+1068*96+1121*108+1145*22+1178*39+1189*39+1209*44+1212*11+1224*66+1257*44+1312*11+1322*32+1377*36+1409*16+1421*10+1450*54+1511*20+1666*21+1753*24+2110*42+2193*95+2285*4+2290*54+3544*12+3673*42+3820*48+3983*18+6280*1+6474*6+6702*14+6960*16+7250*9+7580*4

5399325

# Analysis

## Functions:

In [82]:
def get_Eg_for_single_g(df,numb,power_set_str):
    # print(power_set_str)
    #Obj: puts Eg into correct format to check if topolgoy
    #Inputs: df=Pn, numb used to get a specific function from df
    
    # Pick function in Pn
    g=df.iloc[numb].to_dict()

    #get primitive closed sets
    PC=[(x,get_barj({x},g)) for x in power_set_str]
    dict_PC=dict(PC)
    set_PC=[sorted_frozenset(get_barj({x},g)) for x in power_set_str]

    #Get all closed sets for extensions for g
    Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
    Eg.append(frozenset()) #rembmer the empty set
    return Eg

In [181]:
def get_Eg_for_single_g_dfisfunct(funct,power_set_str):
    # print(power_set_str)
    #Obj: puts Eg into correct format to check if topolgoy
    #Inputs: df=Pn, numb used to get a specific function from df
    
    # Pick function in Pn
    g=funct
    
    print(power_set_str)

    #get primitive closed sets
    PC=[(x,get_barj({x},g)) for x in power_set_str]
    dict_PC=dict(PC)
    set_PC=[sorted_frozenset(get_barj({x},g)) for x in power_set_str]
    # print("len set_PC",len(set_PC))
    
    #Get all closed sets for extensions for g
    Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
    Eg.append(frozenset()) #rembmer the empty set
    return Eg

In [25]:
def get_primitive_for_single_g(df,numb):
    #Inputs: df=Pn, numb used to get a specific function from df
    
    # Pick function in Pn
    g=df.iloc[numb].to_dict()

    #get primitive closed sets
    PC=[(x,get_barj({x},g)) for x in power_set_str]
    dict_PC=dict(PC)
    set_PC=[sorted_frozenset(get_barj({x},g)) for x in power_set_str]


    return dict_PC,set_PC

In [26]:
def is_finite_topological_space(X, T):
    
    """
        # Does this account for unions? 

    # X = {'a', 'b', 'c'}
    # T = [X, set(), {'a'}, { 'c'}, {'b'}]
    # print(is_finite_topological_space(X, T)) # prints False

    # This condition is not met because the union of {'a'} and {'c'} is {'a','c'} which is not in T.
    """
    
    if not all(isinstance(x, set) for x in T):
        # T must be a collection of sets
        print(x)
        
        return False
    if not all(x.issubset(X) for x in T):
        # Each element of T must be a subset of X
        print("here1")

        return False
    if not all(x.intersection(y) in T for x in T for y in T):
        print("here2")

        # The intersection of any two elements of T must be in T
        return False
    if not all(x.union(y) in T for x in T for y in T):
        # The union of any two elements of T must be in T
        print("here3")
        return False
    
    if not X in T:
        print("here4")
        print(x)

        # X must be in T
        return False
    if not set() in T:
        print("here5")

        # The empty set must be in T
        return False
    return True

In [27]:
def check_top_for_single_g(df,numb,power_set_str):
    #Inputs:
    # df=Pn
    # numb=choice of function
    # power_set_str for df
    
    # Output:True or False
    
    Eg=get_Eg_for_single_g(df,numb,power_set_str)
    Top=list(set(x) for x in Eg)
    Top.append(set())
    
    X_Eg=set(power_set_str)
    
    return is_finite_topological_space(X_Eg, Top)

In [28]:
def check_all_funct_topspace(df,power_set_str):
    length=[]
    for i,g in enumerate(df.to_dict("records")): #list of dictionaries of functions
        check_top=check_top_for_single_g(df,i,power_set_str)
        if check_top == False:
            print(f"Function {i} does NOT give a topolgical space: {check_top}")
        else:
            length.append(check_top)
            
    if len(df.to_dict("records"))==len(length):
        print("Every function produces a topological space" )
    else:
        print("something is not a topological space")

In [177]:
def get_Eg_column_Pn(df):
    df=df.copy()
    
    #Get Eg for the row in df.

    # Create a new column called "new_column"
    #--------
    power_set_str_n=get_powerset_str(n+1)
    # print("Power",power_set_str_n)
    print("hii",df)

    # Insert data into the new column
    for i, row in df.iterrows():   
        funct=dict(row)
        funct.pop('Eg_Size', None)
        # print(funct)
        Eg=get_Eg_for_single_g_dfisfunct(funct,power_set_str_n)
        # print(Eg)
        df.at[i, 'Eg_Size'] = int(len(Eg))
        print(len(Eg))
    return df

In [118]:
def eg_column_Build_Pn_1(list_df_Pn,power_set_str,n):
    
    """
    Obj:Use Build_Pn_1 and modify change to include Eg size column in functions
    """
    
    # Returns:packets of extensions for each f_dict from Pn, can check those with max size of closed sets (later)
    
    P_n1=[]
    for index,f_dict in enumerate(list_df_Pn):

        # Store primitive closed sets 
        PC=[(x,get_barj({x},f_dict)) for x in power_set_str]
        dict_PC=dict(PC)
        set_PC=[sorted_frozenset(get_barj({x},f_dict)) for x in power_set_str]

        #Get all closed sets for extensions for f_dict
        Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
        Eg.append("empty")
        
        # #Append eg_size for f_dict
        # power_set_str_n=get_powerset_str(n+1)[0]
        # funct_list=[f_dict] #eg is 5 
        # print(funct_list)
        # f_dict_pd=pd.DataFrame(funct_list)
        # Eg_f_dict=get_Eg_for_single_g(f_dict_pd,0,power_set_str_n)
        Eg_col_kvpair={"Eg_Size":len(Eg)}
        # Eg_col_kvpair={"Eg_Size":len(Eg_f_dict)}

#         print(len(Eg_f_dict))
        
        #Build extensions for f_dict
        extension_of_f_dict=[]
        for term in Eg:
            if term =="empty":                    
                funct=empty_case_get_extension_of_f(f_dict,term,power_set_str,n) #dictionary
                funct.update(Eg_col_kvpair)
                extension_of_f_dict.append(funct)
            else:
                funct=get_extension_of_f(f_dict,term,power_set_str,n)
                funct.update(Eg_col_kvpair)
                extension_of_f_dict.append(funct)

        #We record packets of extensions where we take +1
        P_n1.append(extension_of_f_dict)
    
    return P_n1

In [108]:
def get_Pnp1_for_single_funct(Pn,n):
    
    #inputs
    # Pn:dataframe and n.
    
    """
    Returns a dataframe of extensions in Pn+1 for dataframe of functions in Pn.
    """
    """
    #Example
    n=3
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)
    # P3

    funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]
    Pn=pd.DataFrame(funct_list)

    #The following should be functions in P4 that are extendions of functions in funct_list
    f_P3=get_Pnp1_for_single_funct(Pn,n)
    print(f_P3)
    """

    #New data created
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)

    #Load Pn as a list of dictionaries.
    list_df_Pn=Pn.to_dict(orient='records')

    #Create Pn+1 and hold as dataframe.
    P_n1=eg_column_Build_Pn_1(list_df_Pn,power_set_str,n)
    #As previous was done in E(g) parts, joint together
    Pnp1=extend_sets_to_Pnp1(P_n1)
        
    Pnp1=get_Eg_column_Pn(Pnp1)
    
    columns=list(power_set_str_np1)+["Eg_Size"]
    Pnp1 = Pnp1.reindex(columns=columns)
    
    # inspect_Pn1(Pnp1)

    return Pnp1

In [106]:
def extend_sets_to_Pnp1(P_n1):
    #Return the data frame of Pn+1
    
    final_P_n1=[]
    for ls in P_n1:
        final_P_n1=final_P_n1+ls # Adding all the extension packets together.
    df=pd.DataFrame(final_P_n1)
    # df = df.reindex(columns=power_set_str_np1)
    
    return df

## For $g \in P_3$ get $\mathbb{E}(g)$

The set $\mathbb{E}(g)$ is the topology assoicated to the function $g$ and can be used to construct the extensions of $g$.

We now have:
- A function/method to get $\mathbb{E}(g)$.
- A function to check if $\mathbb{E}(g)$ is a finite topology.
- Check if all function in P3,P4,P5 generate a topology: Answer P3,P4 yes, examples of P5 yes

What we could do next:
- A way to determine the size of $\mathbb{E}(g)$ (using basis).
- Are all Top(g) for $g \in P_n$ homeomorphic? (topologies are different sizes).

### Specific Example from P3 is topological space

In [26]:
power_set_str=get_powerset_str(3)

df=P3
numb=4
#Get data
Eg=get_Eg_for_single_g(df,numb,power_set_str)
# print(power_set_str)
print(P3.iloc[numb].to_dict(),"\n")

for term in Eg:
    print(set(term))
    
print("\n  All closed sets")


{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 0} 

{'12', '123', '23'}
{'12', '1', '123', '13'}
{'123', '3', '23', '13'}
{'3', '2', '23', '12', '1', '123', '13'}
{'123'}
{'3', '23', '12', '123', '13'}
{'2', '23', '12', '1', '123', '13'}
{'12', '123', '23', '13'}
{'12', '123', '13'}
{'23', '12', '1', '123', '13'}
{'12', '23', '123', '2'}
{'12', '123'}
{'123', '23'}
{'3', '23', '2', '12', '123', '13'}
{'123', '13'}
{'23', '2', '12', '123', '13'}
{'123', '23', '13'}
{'3', '23', '12', '1', '123', '13'}
set()

  All closed sets


Check if Eg is a finite topological space on the powerset for an example g in P3

In [ ]:
X_Eg= {'13', '23', '123', '12', '2', '3', '1'} #total space ie powerset
Top=list(set(x) for x in Eg) #topology

print(is_finite_topological_space(X_Eg, Top))

### Primitives for specific example

In [25]:
dict_PC,set_PC=get_primitive_for_single_g(df,numb)

print(P3.iloc[numb].to_dict(),"\n")


# print(dict_PC) #Remembers what the terms of powerset is
set_PC.append(frozenset()) # add empty set: this is the set of primitive closed sets
for i in set_PC:
    print(set(i))
    
print("\n Primitive closed sets")
    
# X_Eg= {'13', '23', '123', '12', '2', '3', '1'} #total space ie powerset
# Prim_Top= list(set(x) for x in set_PC) # primitive set as a topology? is it a basis?

{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 0} 

{'12', '1', '123', '13'}
{'12', '23', '123', '2'}
{'123', '3', '23', '13'}
{'12', '123'}
{'123', '13'}
{'123', '23'}
{'123'}
set()

 Primitive closed sets


### Consider Top(g) for two different g,g' in P3.

In [ ]:
#Setup data
power_set_str=get_powerset_str(3)
df=P3
X_Eg= set(power_set_str) #total space ie powerset

#Inputs
numb1=0
numb2=1

#Top space 1
Eg1=get_Eg_for_single_g(df,numb,power_set_str)
Top1=list(set(x) for x in Eg1) #topology
print(is_finite_topological_space(X_Eg, Top1))
#------------------------------
#Top space 2
Eg2=get_Eg_for_single_g(df,numb2,power_set_str)
Top2=list(set(x) for x in Eg2) #topology
print(is_finite_topological_space(X_Eg, Top2))
#------------------------------



## Get Pn with |Eg| size & Find g's with largest and smallest |Eg|

### P3

In [183]:
power_set_str=get_powerset_str(3)

P3_eg=get_Eg_column_Pn(P3)
P3_eg=P3_eg.sort_values('Eg_Size') # get those 13,then 19
print(P3_eg,"\n")
# print(sum(P3_eg["Eg_Size"]))# check 154 total

df=P3_eg
print(df['Eg_Size'].value_counts()) #count number of 13s and 19s

hii    1  2  3  12  13  23  123
0  0  0  0   0   1   0    1
1  0  0  0   0   0   0    1
2  0  0  0   0   1   1    1
3  0  0  0   0   0   1    1
4  0  0  0   0   0   0    0
5  0  0  0   1   0   1    1
6  0  0  0   1   1   1    2
7  0  0  0   1   1   0    1
8  0  0  0   1   1   1    1
9  0  0  0   1   0   0    1
['1', '2', '3', '12', '13', '23', '123']
13
['1', '2', '3', '12', '13', '23', '123']
19
['1', '2', '3', '12', '13', '23', '123']
13
['1', '2', '3', '12', '13', '23', '123']
13
['1', '2', '3', '12', '13', '23', '123']
19
['1', '2', '3', '12', '13', '23', '123']
13
['1', '2', '3', '12', '13', '23', '123']
19
['1', '2', '3', '12', '13', '23', '123']
13
['1', '2', '3', '12', '13', '23', '123']
19
['1', '2', '3', '12', '13', '23', '123']
13
   1  2  3  12  13  23  123  Eg_Size
0  0  0  0   0   1   0    1     13.0
2  0  0  0   0   1   1    1     13.0
3  0  0  0   0   0   1    1     13.0
5  0  0  0   1   0   1    1     13.0
7  0  0  0   1   1   0    1     13.0
9  0  0  0   1   0   0    

In [ ]:
#Get Eg for the function in P3
#3  0  0  0   0   0   0    1      19
numb =3
df=P3
# g=df.iloc[numb].to_dict()
print(g,"\n")

EEg_example=get_Eg_for_single_g(df,numb,power_set_str)
# for i in EEg_example:
#     print(set(i))

### P4

In [14]:
power_set_str=get_powerset_str(4)

P4_eg=get_Eg_column_Pn(P4)
P4_eg=P4_eg.sort_values('Eg_Size') # get those

# print(P4_eg,"\n")
# print(P4_eg.head) 
print(P4_eg[P4_eg.Eg_Size == 42])

#The sizes of |Eg|
print(P4_eg['Eg_Size'].value_counts()) #count number of 13s and 19s

     1  2  3  4  12  13  14  23  24  34  123  124  134  234  1234 Eg_Size
0    0  0  0  0   0   1   0   0   0   1    1    0    1    1     1      42
141  0  0  0  0   1   0   1   0   1   1    1    2    1    1     2      42
143  0  0  0  0   1   0   1   0   0   0    1    1    1    0     1      42
46   0  0  0  0   0   0   0   1   1   0    1    1    0    1     1      42
119  0  0  0  0   1   1   1   0   0   1    1    1    2    1     2      42
56   0  0  0  0   0   0   1   1   1   1    1    1    1    2     2      42
144  0  0  0  0   1   0   0   0   1   0    1    1    0    1     1      42
35   0  0  0  0   0   1   1   1   0   1    1    1    2    1     2      42
45   0  0  0  0   0   0   0   1   0   1    1    0    1    1     1      42
44   0  0  0  0   0   1   0   1   0   0    1    0    1    1     1      42
67   0  0  0  0   0   0   1   0   0   1    0    1    1    1     1      42
120  0  0  0  0   1   1   1   0   1   0    1    2    1    1     2      42
68   0  0  0  0   0   0   0   0   1   

In [16]:
y=P4_eg[P4_eg.Eg_Size == 42]
print(y)

print(y.shape)


     1  2  3  4  12  13  14  23  24  34  123  124  134  234  1234 Eg_Size
0    0  0  0  0   0   1   0   0   0   1    1    0    1    1     1      42
141  0  0  0  0   1   0   1   0   1   1    1    2    1    1     2      42
143  0  0  0  0   1   0   1   0   0   0    1    1    1    0     1      42
46   0  0  0  0   0   0   0   1   1   0    1    1    0    1     1      42
119  0  0  0  0   1   1   1   0   0   1    1    1    2    1     2      42
56   0  0  0  0   0   0   1   1   1   1    1    1    1    2     2      42
144  0  0  0  0   1   0   0   0   1   0    1    1    0    1     1      42
35   0  0  0  0   0   1   1   1   0   1    1    1    2    1     2      42
45   0  0  0  0   0   0   0   1   0   1    1    0    1    1     1      42
44   0  0  0  0   0   1   0   1   0   0    1    0    1    1     1      42
67   0  0  0  0   0   0   1   0   0   1    0    1    1    1     1      42
120  0  0  0  0   1   1   1   0   1   0    1    2    1    1     2      42
68   0  0  0  0   0   0   0   0   1   

### single function P2,single P3,single P4 what are |Eg| for each list

what is Eg for choose functions that have minimum eg

In [185]:
#Testing functions example
n=2
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
# P2

funct_list=[{'1': 0,'2': 0,"12":0}] #eg is 5 
Pn=pd.DataFrame(funct_list)
print("The functions we are extending")
print(Pn)

#The following should be functions in P4 that are extendions of functions in funct_list
f_P2=get_Pnp1_for_single_funct(Pn,n)
print(f_P2)

The functions we are extending
   1  2  12
0  0  0   0
hii    1  2  12  123  13  23  3  Eg_Size
0  0  0   0    1   1   0  0        5
1  0  0   0    1   0   0  0        5
2  0  0   0    1   1   1  0        5
3  0  0   0    1   0   1  0        5
4  0  0   0    0   0   0  0        5
['1', '2', '3', '12', '13', '23', '123']
27
['1', '2', '3', '12', '13', '23', '123']
27
['1', '2', '3', '12', '13', '23', '123']
21
['1', '2', '3', '12', '13', '23', '123']
27
['1', '2', '3', '12', '13', '23', '123']
37
   1  2  3  12  13  23  123  Eg_Size
0  0  0  0   0   1   0    1       27
1  0  0  0   0   0   0    1       27
2  0  0  0   0   1   1    1       21
3  0  0  0   0   0   1    1       27
4  0  0  0   0   0   0    0       37


In [35]:
#Testing functions example
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
# P3

# funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]

funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":1,"123":1}]

# funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]

Pn=pd.DataFrame(funct_list)
print("The functions we are extending")
print(Pn)

#The following should be functions in P4 that are extendions of functions in funct_list
f_P3=get_Pnp1_for_single_funct(Pn,n)
print(f_P3)

The functions we are extending
   1  2  3  12  23  13  123
0  0  0  0   0   0   1    1
Number of functions: 13
Are there duplicates? (0, 16) if (0,-) then no common rows
    1  2  3  4  12  13  14  23  24  34  123  124  134  234  1234  Eg_Size
0   0  0  0  0   0   1   0   0   0   1    1    0    1    1     1       13
1   0  0  0  0   0   1   1   0   0   0    1    1    1    0     1       13
2   0  0  0  0   0   1   1   0   0   1    1    1    2    1     2       13
3   0  0  0  0   0   1   0   0   0   0    1    1    1    0     1       13
4   0  0  0  0   0   1   1   0   0   1    1    1    1    1     2       13
5   0  0  0  0   0   1   0   0   0   0    1    1    1    1     1       13
6   0  0  0  0   0   1   1   0   0   1    1    1    1    1     1       13
7   0  0  0  0   0   1   1   0   0   0    1    1    1    1     1       13
8   0  0  0  0   0   1   0   0   0   0    1    0    1    1     1       13
9   0  0  0  0   0   1   0   0   0   1    1    1    1    1     1       13
10  0  0  0  0  

## Assuming the zero function gives max Eg what is Eg


In [ ]:
power_set_str=get_powerset_str(5)


#Find index of zero function

# Take a g in P5 and calculate Eg
# P5.iloc[1].to_dict()
all_zero_funct={'1': 0,'2': 0,'3': 0,'4': 0,'5': 0,'12': 0,'13': 0,'14': 0,'15': 0,'23': 0,'24': 0,'25': 0,'34': 0,'35': 0,'45': 0,'123': 0,'124': 0,'125': 0,'134': 0,'135': 0,'145': 0,'234': 0,'235': 0,'245': 0,'345': 0,'1234': 0,'1235': 0,'1245': 0,'1345': 0,'2345': 0,'12345': 0}
df=P5
my_dict=all_zero_funct
index = df.loc[df.eq(my_dict).all(1)].index[0]
print(index)#5166 #works
# df.loc[5166]#
# Get Eg for this. #Should be maximum
# Eg_zeroP5=get_Eg_for_single_g(df,index,power_set_str)
# print(len(Eg_zeroP5)) #7580

"""Assuming the "Something" function gives min Eg, what is Eg
not the min or max function, something else.
"""

## Check if all function in P3,P4,P5 generate a topological space: Yes

#### P3

In [ ]:
#inputs
df=P3
power_set_str=get_powerset_str(3)
check_all_funct_topspace(df,power_set_str)

#### P4

In [ ]:
#inputs
df=P4
power_set_str=get_powerset_str(4)
check_all_funct_topspace(df,power_set_str) #False

#### P5 (where Eg for a function allow for calc of part of P6)

In [ ]:
#Take long: 

#inputs
df=P5
power_set_str=get_powerset_str(5)
check_all_funct_topspace(df,power_set_str)

In [ ]:
#Works for some choosen example.

df=P5
power_set_str=get_powerset_str(5)
numb=0 # function indexed at 0

Eg=get_Eg_for_single_g(df,numb,power_set_str) # set of closed sets
X_Eg= set(power_set_str) #total space ie powerset
Top=list(set(x) for x in Eg) #topology from closed sets Eg #Size: 1178

print(len(Top))

#Check whether topological space:
print(is_finite_topological_space(X_Eg, Top)) #The total powerset on 4 elements is not included.

#Result:
#numb=0
#1178 #Top size
#True

# Leftover code

In [ ]:
Question chatgpt:
    
let X be the set {'13', '23', '123', '12', '2', '3', '1'},

and the topology be:
{
{'13', '23', '123', '1', '2'},
{'13', '23', '123', '12', '2', '3', '1'},
{'13', '1'},
{'13', '2', '23', '1'},
{'13', '2', '23'},
{'13', '23'},
{'13', '23', '123', '12', '2', '1'},
{'23'},
{'13'},
{'13', '23', '123', '1', '2', '3'},
{'13', '23', '1'},
{'23', '2'},
set() }

what properties does this topological space have?

This topological space "is":
- Hausdorff,Compact,Second Countable,T0,T1,Regular,Normal

This topological space "is not":
- Discrete: not all subsets of X are open, for example the subsets {'12'} and {'3'} are not open in the topology T.

## Examples for understanding code

Are primitive closed sets a basis for the topology? (Maybe not relevant)

In [ ]:

Asides:
- Is Eg a matriod? No does not satisfy downward closure. In our sepcific example f(12)=1,f(123)=1 else 0 Take {1} in the closure of 1 i.e {1,13}, {1} is not a closed set.
- Are primite closed sets a basis for the topology? Is there a way to construct $\mathbb{E}(g)$ using primitive (using minimal basis).

In [ ]:
# dict_PC,set_PC=get_primitive_for_single_g(df,numb)

# # print(dict_PC) #Remembers what the terms of powerset is
# set_PC.append(frozenset()) # add empty set: this is the set of primitive closed sets
# # for i in set_PC:
# #     print(set(i))
    
# X_Eg= {'13', '23', '123', '12', '2', '3', '1'} #total space ie powerset
# Prim_Top= list(set(x) for x in set_PC) # primitive set as a topology? is it a basis?

# def is_basis(X, T, B):
#     if not all(isinstance(x, set) for x in B):
#         # B must be a collection of sets
#         return False
#     if not all(x.issubset(X) for x in B):
#         # Each element of B must be a subset of X
#         return False
    
    
#     if not all(x.intersection(y) == set() for x in B for y in B if x != y):
#         # The intersection of any two distinct elements of B must be empty
#         return False
    
#     if not all(x.union(y) in T for x in B for y in B):
#         # The union of any two elements of T must be in T
#         return False
    
#     if not all(x.issubset(y) or y.issubset(x) for x in B for y in T):
#         # Each element of B must be a subset of some element of T or conversely
#         return False
#     return True

# # Example usage
# X = {'a', 'b', 'c'}
# T = [X, set(), {'a', 'b'}, {'b', 'c'}, {'b'}]
# B = [{'a', 'b'}, {'b', 'c'}]
# print(is_basis(X, T, B)) # prints True


In [ ]:
#Get all functions in P3 stored a data frame

#Construct P3
pre_functs=[["12","123"],["13","123"],["23","123"],["12","13","123"],["12","23","123"],["13","23","123"],["12","13","23","123"],[],["123"]]
ds=[]
for vals in pre_functs:
    builder1=[(x,1) for x in vals]
    builder0=[(x,0) for x in power3 if x not in vals]
    f_dict={**dict(builder1),**dict(builder0)}
    ds.append(f_dict)
big={'123': 2, '1': 0, '2': 0, '3': 0, '12': 1, '13': 1, '23': 1} #remaining case, with 2 value
dp=ds+[big,]

#Build dataframe
P3 = pd.DataFrame(dp)

#order columns of P3
power3=get_powerset_str(3) #powerset on 3 elements #for 
column_order=power3 #as strings
P3 = P3.reindex(columns=column_order)

print(P3)

In [ ]:
## Checking if the calculations for P5, are the same as our old calculations for P5 in xlsx.

# #Previous data P5 in correct format
# dataframe2 = pd.read_excel('P4_P5_xlsx\P_5.xlsx')

# #Make sure columns are in the same order as P5

# #Reordering columns
# column_order=[int(x) for x in power_set_str_np1]
# dataframe2 = dataframe2.reindex(columns=column_order)

# #Making sure columns are the same, in string format
# dataframe2=string_cols(dataframe2)

# # print(dataframe2.head)
# print(dataframe2.shape) #(10334, 31)

# # n=4
# # power_set_str,alt_power_set_str_np1=get_n_np1_powersets(n)
# # check_functs_MSA(dataframe2,power_set_str_np1).shape #(0,0) i.e all are msa

# Comparision of P5 and dataframe 2. We see they are the same.

# compar=check_df_duplicates(P5,dataframe2,"left",True) #True:10334 , False:0

In [ ]:
# # The calculations for P4, are the same as our old calculations for P4 in xlsx.

# dataframe1 = pd.read_excel('P4_P5_xlsx\P_4.xlsx')
# #Making sure columns are the same, in string format and order as P4 above
# dataframe1=string_cols(dataframe1)

# # #Comparision of P4 and xlsx version
# compar=check_df_duplicates(P4,dataframe1,"left",True)

In [ ]:
# n=3
# # Create a set with 3 elements
# # s = {1, 2, 3}
# s=set(range(1,n+1))

# # Generate all possible subsets of the set
# power_set = [set(x)  for r in range(len(s) + 1) for x in itertools.combinations(s, r)]

# # Print the power set
# print(power_set)

# power_set_str=[set_to_string(x) for x in power_set]
# power_set_str=[set_to_string(x) for x in power_set if len(x)>0]
# power_set_str

# #Get a function in P3

# val1_loc=["12","123"] #where it take 1
# val0_loc= [x for x in power_set_str if x not in val1_loc] #where it takes 0
# # print(val0_loc)

# builder1=[(x,1) for x in val1_loc]
# builder0=[(x,0) for x in val0_loc]
# builder0
# dict(builder0)
# dict(builder1)
# f_dict={**dict(builder1),**dict(builder0)}
# f_dict

# A,B="1","123"
# print(f"Is {A} subset of {B}, f minimal? : {fmin(A,B,f_dict)}")
# print(f"Is {A} subset of {B}, f maximal? : {fmax(A,B,f_dict)}")

#Testing for obtaining primitive sets:
# #Example
# for x in power_set_str:
#     print(x,get_barj({x},f_dict))

#As all the closed sets, checked by hand:
# 3 T term {'13', '123', '1', '3', '23', '2'}
# 1 {'1', '13'}
# 2 {'23', '2'}
# 12 {'1', '12', '123', '13', '2', '23'} 
# 13 {'13'} # should be just 13
# 23 {'23'} # should be 23
# 123 {'13', '123', '1', '23', '2'} # should not have 3 or 12

# Store primitive closed sets 

# PC=[(x,get_barj({x},f_dict)) for x in power_set_str]
# dict_PC=dict(PC)
# set_PC=[sorted_frozenset(get_barj({x},f_dict)) for x in power_set_str]
# # set(set_PC)
# Eg={sorted_frozenset(power_set_str)}.union(set_PC)

# Using main functions:
# Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
# # print(len(x)) # I get like 13 so this is an issue.
# Eg
# Eg.append("empty")
# Eg


#Rows are not in the same order:
# # print(P4.equals(dataframe1))

#Checking if duplicates, #https://stackoverflow.com/questions/48647534/find-difference-between-two-data-frames?noredirect=1&lq=1
# df = P4.merge(dataframe1, how = 'inner' ,indicator=False)
""" 
# first dataframe
df1 = pd.DataFrame({
    'Age': ['20', '14', '56', '28', '10'],
    'Weight': [59, 29, 73, 56, 48]})
display(df1)

# second dataframe
df2 = pd.DataFrame({
    'Age': ['16', '20', '24', '40', '22'],
    'Weight': [55, 59, 73, 85, 56]})
display(df2)

#Common rows:
df = df1.merge(df2, how = 'inner' ,indicator=False)
print(df)
"""

"The above method only works for those data frames that don't already have duplicates themselves. For example:"
no_dups=pd.concat([P4,dataframe1]).drop_duplicates(keep=False)
no_dups.head()
